<a href="https://colab.research.google.com/github/akimi-yano/MLE-MiniProjects/blob/main/MLE_MiniProject_Recommendation_Engines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [283]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).

In [284]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [285]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?
print(movies)
print(movies.shape)
print(movies.nunique())

print('*'*100)

print(ratings)
print(ratings.shape)
print(ratings.nunique())

      movie_id                                      title release_date  \
0            1                           Toy Story (1995)  01-Jan-1995   
1            2                           GoldenEye (1995)  01-Jan-1995   
2            3                          Four Rooms (1995)  01-Jan-1995   
3            4                          Get Shorty (1995)  01-Jan-1995   
4            5                             Copycat (1995)  01-Jan-1995   
...        ...                                        ...          ...   
1677      1678                          Mat' i syn (1997)  06-Feb-1998   
1678      1679                           B. Monkey (1998)  06-Feb-1998   
1679      1680                       Sliding Doors (1998)  01-Jan-1998   
1680      1681                        You So Crazy (1994)  01-Jan-1994   
1681      1682  Scream of Stone (Schrei aus Stein) (1991)  08-Mar-1996   

      video_release_date                                           imdb_url  \
0                    NaN  http:/

In [286]:
# Merge movies and ratings dataframes
df = pd.merge(movies, ratings, on='movie_id')
df.index = df['movie_id']
df = df[['user_id', 'title', 'rating']]
df.head()

,user_id,title,rating
movie_id,,,
1,308,Toy Story (1995),4
1,287,Toy Story (1995),5
1,148,Toy Story (1995),4
1,280,Toy Story (1995),4
1,66,Toy Story (1995),3


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [287]:
def get_movie_title_by_id(movie_id):
  return movies[movies['movie_id'] == movie_id]['title']

In [288]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df):
  # Get the user's rated movies
  user_ratings = df[df['user_id'] == user_id]
  rated_movies_ids = set(user_ratings.index)

  # Create a TF-IDF matrix using movie genres
  genre_matrix = movies[genre_cols]
  genre_matrix.index = movies['movie_id']
  # print(genre_matrix)
  # print(genre_matrix.shape)

  # Create TF-IDF vectorizer and fit to genre matrix
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_vectorized_data = tfidf_vectorizer.fit_transform(genre_matrix.apply(lambda x: ' '.join(x.index[x==1]), axis=1))
  # print(tfidf_vectorized_data)
  # print(tfidf_vectorized_data.shape)
  tfidf_df = pd.DataFrame(tfidf_vectorized_data.toarray(), index=genre_matrix.index, columns=tfidf_vectorizer.get_feature_names_out())
  # print("TFIDF: ", tfidf_df.shape, tfidf_df)

  # Compute the cosine similarity between movie genres
  cosine_sim = cosine_similarity(tfidf_df)
  cosine_sim_df = pd.DataFrame(cosine_sim, index=tfidf_df.index, columns=tfidf_df.index)
  # print("COSINE SIM: ", cosine_sim_df.shape, cosine_sim_df)

  # Get the indices of the similar movies based on cosine similarity
  all_ids_of_similar_movies = set([])
  for movie_id in rated_movies_ids:

    # if movie_id in cosine_sim_df.index:
    #   print(f"The movie '{movie_id}' is in the DataFrame.")
    # else:
    #   print(f"The movie '{movie_id}' is NOT in the DataFrame.")

    cosine_similarity_series = cosine_sim_df.loc[movie_id, :]
    # print(movie_id, type(cosine_similarity_series), cosine_similarity_series)
    ordered_simiarity_ids_by_cosine_sim = cosine_similarity_series.sort_values(ascending=False).index
    top_5_ids = ordered_simiarity_ids_by_cosine_sim[:5]
    # print(movie_id, top_5_ids)

    # contant number 5 of itereation to add the title to the set
    for id in top_5_ids:
      all_ids_of_similar_movies.add(id)

  # Remove duplicates and movies already rated by the user
  # Used the set data structure for all_ids_of_similar_movies to automatically remove duplicates and removes a move at a constant time
  # Just I am just keeping top 5s for each previous reviewed movie all combined together, I decided to not worry about the order
  # However, if we want to keep the original order of the recommendation, set is not a good way as it does not keep the order.
  # print(type(all_ids_of_similar_movies), all_ids_of_similar_movies)
  # print(type(rated_movies_ids), rated_movies_ids)
  all_ids_of_similar_movies = all_ids_of_similar_movies - rated_movies_ids
  # print(all_titles_of_similar_movies)

  recommended_movies_titles = []
  for id in all_ids_of_similar_movies:
    recommended_movies_titles.append(get_movie_title_by_id(id))

  # print("HERE ARE THE CONTENT BASED RECOMMENDATIONS: ", recommended_movies_titles)
  return recommended_movies_titles
# print(df['user_id'].value_counts())

# Testing with some users :)
content_based_recommendation(895, df)
# content_based_recommendation(308, df)

# Notes: we could potentially clean up the structure of the code a little as not all the codes above has to be run at the same frequency and timing.
# For example, the TF-IDF code and cosine similarity code do not necesarily have to run everytime a user signs in to the account.
# Maybe it can be updated once they add a new review or any action to update the User Profile.


[767    Casper (1995)
 Name: title, dtype: object,
 1    GoldenEye (1995)
 Name: title, dtype: object,
 1411    Land Before Time III: The Time of the Great Gi...
 Name: title, dtype: object,
 4    Copycat (1995)
 Name: title, dtype: object,
 1027    Grumpier Old Men (1995)
 Name: title, dtype: object,
 1672    Mirage (1995)
 Name: title, dtype: object,
 1161    Phat Beach (1996)
 Name: title, dtype: object,
 270    Starship Troopers (1997)
 Name: title, dtype: object,
 273    Sabrina (1995)
 Name: title, dtype: object,
 1172    Bliss (1997)
 Name: title, dtype: object,
 665    Blood For Dracula (Andy Warhol's Dracula) (1974)
 Name: title, dtype: object,
 1049    Ghost and Mrs. Muir, The (1947)
 Name: title, dtype: object,
 668    Body Parts (1991)
 Name: title, dtype: object,
 1567    Vermont Is For Lovers (1992)
 Name: title, dtype: object,
 32    Desperado (1995)
 Name: title, dtype: object,
 929    Chain Reaction (1996)
 Name: title, dtype: object,
 1313    Surviving the Game (1994)

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [289]:
# Collaborative Filtering using User-Item Interactions
def collaborative_filtering_recommendation(user_id, df):
  # Create the user-item matrix
  user_item_matrix = df.pivot_table(index='user_id', columns=df.index, values='rating')
  # print(user_item_matrix)

  # Fill missing values with 0 (indicating no rating)
  user_item_matrix = user_item_matrix.fillna(0)
  # print(user_item_matrix)

  # Calculate user-user similarity matrix using cosine similarity
  user_similarity = cosine_similarity(user_item_matrix)
  user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
  # print(user_similarity_df)

  # Get the similarity scores of the target user with all other users
  target_user_similarity = user_similarity_df.loc[user_id]
  # print(target_user_similarity)

  # Find the top N most similar users (excluding the target user)
  top_n = 5
  top_n_similar_users = target_user_similarity.sort_values(ascending=False).index[1:top_n+1]
  # print(user_id, top_n_similar_users)
  # print(top_n_similar_users)

  # Generate movie recommendations based on the most similar users
  all_ids_of_recommended_movies = set([])
  for user in top_n_similar_users:
    user_ratings = user_item_matrix.loc[user]
    # print(user, " reviewed the following movies: ", user_ratings)

    # Sort and use the movies whose rates are higher than the threshold
    rating_threshold = 3
    sorted_user_ratings = user_ratings.sort_values(ascending=False)
    # print(user, " rated the movies in the following order: ", sorted_user_ratings)
    sorted_movies_higher_than_threshold = sorted_user_ratings[sorted_user_ratings > rating_threshold]
    # print(user, " highly rated the following movies in the following order: ", sorted_movies_higher_than_threshold)
    top_m = 5
    top_m_sorted_movies_higher_than_threshold = sorted_movies_higher_than_threshold[:5]
    # print(user, " highly rated the m following movies: ", top_m_sorted_movies_higher_than_threshold)

    for movie in sorted_movies_higher_than_threshold.index:
      all_ids_of_recommended_movies.add(movie)

  # print(recommendations)

  # Remove duplicates from recommendations
  # Used the set as a data structure to remove duplicates automatially

  recommended_movies_titles = []
  for id in all_ids_of_recommended_movies:
    recommended_movies_titles.append(get_movie_title_by_id(id))

  # print("HERE ARE THE COLLABORATIVE FILTERING RECOMMENDATIONS: ", recommended_movies_titles)
  return recommended_movies_titles

# Testing with a user :)
collaborative_filtering_recommendation(895, df)

[1151    In Love and War (1996)
 Name: title, dtype: object,
 0    Toy Story (1995)
 Name: title, dtype: object,
 257    Contact (1997)
 Name: title, dtype: object,
 256    Men in Black (1997)
 Name: title, dtype: object,
 6    Twelve Monkeys (1995)
 Name: title, dtype: object,
 263    Mimic (1997)
 Name: title, dtype: object,
 267    Chasing Amy (1997)
 Name: title, dtype: object,
 268    Full Monty, The (1997)
 Name: title, dtype: object,
 13    Postino, Il (1994)
 Name: title, dtype: object,
 14    Mr. Holland's Opus (1995)
 Name: title, dtype: object,
 274    Sense and Sensibility (1995)
 Name: title, dtype: object,
 404    Mission: Impossible (1996)
 Name: title, dtype: object,
 150    Willy Wonka and the Chocolate Factory (1971)
 Name: title, dtype: object,
 280    River Wild, The (1994)
 Name: title, dtype: object,
 1176    Dunston Checks In (1996)
 Name: title, dtype: object,
 537    Anastasia (1997)
 Name: title, dtype: object,
 282    Emma (1996)
 Name: title, dtype: object,


Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

In [290]:
# Test the recommendation engines
test_users = [308, 287, 148, 280, 66]
for test_user_id in test_users:
  print("User ID: ", test_user_id)
  print("Content Based Recommendations: ", content_based_recommendation(test_user_id, df))
  print("Collaborative Filtering Recommendations: ", collaborative_filtering_recommendation(test_user_id, df))
  print("*"*100)

# Notes:
# It is interesting that the recommended results are so different between the two methods!
# It does make sense though it would be difficult to validate as the recommendations are done for the movies that the users have not watched yet.

# Take aways:
# Use the movie_id (which I saved as index) instead of titles to make sure they are unique to avoid unexpected types between DataFrame and Series.

User ID:  308
Content Based Recommendations:  [1024    Fire Down Below (1997)
Name: title, dtype: object, 1    GoldenEye (1995)
Name: title, dtype: object, 1033    Quest, The (1996)
Name: title, dtype: object, 1035    Drop Dead Fred (1991)
Name: title, dtype: object, 1036    Grease 2 (1982)
Name: title, dtype: object, 1552    Underneath, The (1995)
Name: title, dtype: object, 1041    Just Cause (1995)
Name: title, dtype: object, 1555    Condition Red (1995)
Name: title, dtype: object, 532    Daytrippers, The (1996)
Name: title, dtype: object, 1556    Yankee Zulu (1994)
Name: title, dtype: object, 537    Anastasia (1997)
Name: title, dtype: object, 538    Mouse Hunt (1997)
Name: title, dtype: object, 1049    Ghost and Mrs. Muir, The (1947)
Name: title, dtype: object, 28    Batman Forever (1995)
Name: title, dtype: object, 540    Mortal Kombat (1995)
Name: title, dtype: object, 542    Misérables, Les (1995)
Name: title, dtype: object, 543    Things to Do in Denver when You're Dead (1995)

In [291]:
df[df['user_id'] == 895]

,user_id,title,rating
movie_id,,,
1,895,Toy Story (1995),4
13,895,Mighty Aphrodite (1995),5
50,895,Star Wars (1977),5
100,895,Fargo (1996),4
117,895,"Rock, The (1996)",3
151,895,Willy Wonka and the Chocolate Factory (1971),5
181,895,Return of the Jedi (1983),5
222,895,Star Trek: First Contact (1996),3
275,895,Sense and Sensibility (1995),5
